In [2]:
import pandas as pd

import os
from dotenv import load_dotenv, dotenv_values, set_key
# import pathlib

from os.path import join, dirname

path = 'D:\\Users\\hatem.kamal\\Desktop\\DE udacity\\Test\\CassandraPractice\\'
dotenv_path = join(path, '.env')

load_dotenv(dotenv_path)

CR_Token = os.getenv(f'CR_Token')
adm_token = os.getenv(f'ADM_TOKEN')
ASTRA_DB_SECURE_BUNDLE_PATH = os.getenv(f'ASTRA_DB_SECURE_BUNDLE_PATH')
DB_ID = os.getenv(f'DB_ID')

In [13]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

# with open(ASTRA_TOKEN_PATH, "r") as f:
#     creds = json.load(f)
    # ASTRA_DB_APPLICATION_TOKEN = creds["token"]

ASTRA_DB_APPLICATION_TOKEN = adm_token
try: 
    cluster = Cluster(
        cloud={
            "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
        },
        auth_provider=PlainTextAuthProvider(
            "token",
            ASTRA_DB_APPLICATION_TOKEN,
        ),
    )

    session = cluster.connect()


    session = cluster.connect()
except Exception as e:
    print(e)

In [14]:
try:
    session.set_keyspace('project_keyspace')
except Exception as e:
    print(e)

### Walk through the basics of creating a table in Apache Cassandra, inserting rows of data, and doing a simple CQL query to validate the information. You will practice Denormalization, and the concept of 1 table per query, which is an encouraged practice with Apache Cassandra. 

### Remember, replace ##### with your answer.

### Let's imagine we would like to start creating a Music Library of albums. 

### We want to ask 3 questions of the data
#### 1. Give every album in the music library that was released in a given year
`select * from music_library WHERE YEAR=1970`
#### 2. Give every album in the music library that was created by a given artist  
`select * from artist_library WHERE artist_name="The Beatles"`
#### 3. Give all the information from the music library about a given album
`select * from album_library WHERE album_name="Close To You"`


In [7]:
def exec_query(query, session):
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [8]:
create_query = "CREATE TABLE IF NOT EXISTS "
query = create_query + "music_library (year int, artist_name varchar, album_name varchar, PRIMARY KEY (year, album_name))"
exec_query(query, session)

query1 = create_query + "artist_library (year int, artist_name varchar, album_name varchar, PRIMARY KEY (artist_name, album_name))"
exec_query(query1, session)

query2 = create_query + "album_library (year int, artist_name varchar, album_name varchar, PRIMARY KEY (album_name, artist_name))"
exec_query(query2, session)

In [10]:
insert_query = "INSERT INTO "
query = insert_query + "music_library (year, artist_name, album_name) VALUES (%s, %s, %s)"

query1 = "INSERT INTO "
query1 = insert_query + "artist_library (artist_name,year,  album_name) VALUES (%s, %s, %s)"

query2 = "INSERT INTO "
query2 = insert_query + "album_library (album_name,artist_name,  year) VALUES (%s, %s, %s)"

try:
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Who", "My Generation"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1966, "The Monkees", "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query, (1970, "The Carpenters", "Close To You"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Beatles", 1970, "Let it Be"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Beatles", 1965, "Rubber Soul"))
except Exception as e:
    print(e)
    
try:
    session.execute(query1, ("The Who", 1965, "My Generation"))
except Exception as e:
    print(e)

try:
    session.execute(query1, ("The Monkees", 1966, "The Monkees"))
except Exception as e:
    print(e)

try:
    session.execute(query1, ("The Carpenters", 1970, "Close To You"))
except Exception as e:
    print(e)
    
try:
    session.execute(query2, ("Let it Be", "The Beatles", 1970))
except Exception as e:
    print(e)
    
try:
    session.execute(query2, ("Rubber Soul", "The Beatles", 1965))
except Exception as e:
    print(e)
    
try:
    session.execute(query2, ("My Generation", "The Who", 1965))
except Exception as e:
    print(e)

try:
    session.execute(query2, ("The Monkees", "The Monkees", 1966))
except Exception as e:
    print(e)

try:
    session.execute(query2, ("Close To You", "The Carpenters", 1970))
except Exception as e:
    print(e)

In [11]:
query = "SELECT * FROM music_library WHERE year=1970"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.year, row.artist_name, row.album_name)

1970 The Carpenters Close To You
1970 The Beatles Let it Be


In [15]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table album_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "drop table artist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [16]:
session.shutdown()
cluster.shutdown()